In [ ]:
%pip install -q google-generativeai ipywidgets

In [ ]:
import os
import getpass
import dotenv
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import google.generativeai as palm

In [ ]:
palm.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [ ]:
models = [
    m for m in palm.list_models() if "generateText" in m.supported_generation_methods
]

for m in models:
    print(f"Model Name: {m.name}")

Embeddings and Vector Stores

In [ ]:
%pip install pypdf
%pip install faiss-cpu
%pip install -U langchain-google-genai

In [ ]:
import os
import getpass
import dotenv
import langchain
import langchain_experimental
import google.generativeai
import pypdf
from dotenv import load_dotenv

In [ ]:
from langchain.llms import google_palm
from langchain.vectorstores import FAISS
from langchain.embeddings import GooglePalmEmbeddings
from langchain.llms import GooglePalm
from langchain.document_loaders import PyPDFLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings

load_dotenv()

In [ ]:
loader = PyPDFLoader("./data/FTA PMO RFP_Final_11.09.23_Executed.pdf")
pages = loader.load_and_split()

In [ ]:
pages[0]

In [ ]:
faiss_index = FAISS.from_documents(pages, GooglePalmEmbeddings(model_name="models/embedding-gecko-001",google_api_key=os.getenv("GOOGLE_API_KEY")))

In [ ]:
docs = faiss_index.similarity_search("what are the duties of the program manager", k=5)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content[:300])

Implement RAG Fusion

In [ ]:
import os
import getpass
import dotenv
import langchain
import langchain_experimental
import google.generativeai as palm
from dotenv import load_dotenv
load_dotenv()


In [ ]:
from langchain.llms import google_palm
from langchain.llms import openai
from langchain.vectorstores import FAISS
from langchain.embeddings import GooglePalmEmbeddings
from langchain.llms import GooglePalm
from langchain.document_loaders import PyPDFLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
%pip install -U langsmith langchainhub
%pip install tiktoken

In [ ]:
from langchain.callbacks.manager import tracing_v2_enabled
from langsmith import Client
lsclient = Client()

export LANGCHAIN_TRACING_V2=true
export LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
export LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")  # if not specified, defaults to "default"
export OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

In [ ]:
embeddings=GooglePalmEmbeddings(model_name="models/embedding-gecko-001",google_api_key=os.getenv("GOOGLE_API_KEY"))

In [ ]:
loader = PyPDFLoader("./data/FTA PMO RFP_Final_11.09.23_Executed.pdf")
documents = loader.load_and_split()

In [ ]:
# Get your splitter ready
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=300)

# Split your docs into texts
docs = text_splitter.split_documents(documents)

In [ ]:
db = FAISS.from_documents(docs, embeddings)

In [ ]:
retriever = db.as_retriever(search_type="mmr", search_kwargs={'k': 6, 'lambda_mult': 0.25})

In [ ]:
import openai

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
def generate_queries_chatgpt(original_query):
    response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[
            {"role": "system", "content": "Your task is to generate multiple different search queries that aim to answer the user question from multiple perspectives. Each query MUST tackle the question from a different viewpoint, we want to get a variety of RELEVANT search results. Each query MUST be in one line and one line only. You SHOULD NOT include any preamble or explanations, and you SHOULD NOT answer the questions or add anything else, just geenrate the queries."},
            {"role": "user", "content": f"Generate multiple search queries related to: {original_query}"},
            {"role": "user", "content": "OUTPUT (5 queries):"}
        ]
    )

    generated_queries = response.choices[0].message.content.strip().split("\n")
    return generated_queries

def vector_search(query):
    search_results = {}
    retrieved_docs = retriever.get_relevant_documents(query)
    for i in retrieved_docs:
        search_results[i.page_content] = i.metadata['source']
    return search_results

def reciprocal_rank_fusion(search_results_dict, k=60):
    fused_scores = {}
        
    for query, doc_scores in search_results_dict.items():
        
        for rank, (doc, score) in enumerate(sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)):
            if doc not in fused_scores:
                fused_scores[doc] = 0
            previous_score = fused_scores[doc]
            fused_scores[doc] += 1 / (rank + k)
            print(f"Updating score for {doc} from {previous_score} to {fused_scores[doc]} based on rank {rank} in query '{query}'")

    reranked_results = {doc: score for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)}
    print("Final reranked results:", reranked_results)
    return reranked_results


def generate_output(original_query, reranked_results):
    reranked_docs = [i for i in reranked_results.keys()]
    context = '\n'.join(reranked_docs)
    response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that answers user's questions based on the context provided.\nDo not make up an answer if you do not know it, stay within the bounds of the context provided, if you don't know the answer, say that you don't have enough information on the topic!"},
            {"role": "user", "content": f"CONTEXT: {context}\nQUERY: {original_query}"},
            {"role": "user", "content": "ANSWER:"}
        ]
    )

    response = response.choices[0].message.content.strip()
    return response

In [ ]:
original_query = "what type of resume would be most sucessful to fill the role of Program Manager for this RFP?"
generated_queries = generate_queries_chatgpt(original_query)

all_results = {}
for query in generated_queries:
    search_results = vector_search(query)
    all_results[query] = search_results

reranked_result = reciprocal_rank_fusion(all_results)
final_output = generate_output(original_query, reranked_result)

In [ ]:
final_output

In [ ]:
print(generated_queries)